# Reports

In [7]:
import os
from datetime import date

import pandas as pd
import matplotlib.pyplot as plt

# So plots show inline in Jupyter
%matplotlib inline

AWS_PROFILE = "de_jay_east"
REGION = "us-east-1"

LANDING_BUCKET = "dq-landing-657082399901-dev"
PROCESSED_BUCKET = "dq-processed-657082399901-dev"
ANALYTICS_BUCKET = "dq-analytics-657082399901-dev"

# If you're using awscli profile, configure env for s3fs/boto
os.environ["AWS_PROFILE"] = AWS_PROFILE
os.environ["AWS_DEFAULT_REGION"] = REGION

today = date.today().isoformat()
today


'2025-12-19'

In [8]:
import pandas as pd

ts_path = f"s3://{PROCESSED_BUCKET}/processed/bls/time_series_pr/"
pop_path = f"s3://{PROCESSED_BUCKET}/processed/population/population_us/"

df_ts = pd.read_parquet(
    ts_path,
    engine="pyarrow",
    dtype_backend="pyarrow",   # <-- key part
)

df_pop = pd.read_parquet(
    pop_path,
    engine="pyarrow",
    dtype_backend="pyarrow",
)

df_ts.head(), df_ts.dtypes


(     series_id period   value  year
 0  PRS84006012    Q02    -1.4  1947
 1  PRS84006012    Q03     4.6  1947
 2  PRS84006012    Q04     3.6  1947
 3  PRS84006013    Q01  42.414  1947
 4  PRS84006013    Q02   42.26  1947,
 series_id                                      string[pyarrow]
 period                                         string[pyarrow]
 value                                          double[pyarrow]
 year         dictionary<values=int32, indices=int32, ordere...
 dtype: object)

In [3]:
df_pop.head(), df_pop.dtypes


(   population  year
 0   316128839  2013
 1   318857056  2014
 2   321418821  2015
 3   323127515  2016
 4   325719178  2017,
 population                                       int64[pyarrow]
 year          dictionary<values=int32, indices=int32, ordere...
 dtype: object)

In [10]:
import s3fs

fs = s3fs.S3FileSystem(profile=AWS_PROFILE)
run_date = "2025-11-24"  # or whatever date you want

best_prefix = (
    f"{ANALYTICS_BUCKET}/analytics/reports/"
    f"run_date={run_date}/report_best_year.csv/"
)
joined_prefix = (
    f"{ANALYTICS_BUCKET}/analytics/reports/"
    f"run_date={run_date}/report_joined.csv/"
)

print("Best year prefix:", best_prefix)
print("Joined prefix:", joined_prefix)

best_files = fs.ls(best_prefix)
joined_files = fs.ls(joined_prefix)

best_files, joined_files


Best year prefix: dq-analytics-657082399901-dev/analytics/reports/run_date=2025-11-24/report_best_year.csv/
Joined prefix: dq-analytics-657082399901-dev/analytics/reports/run_date=2025-11-24/report_joined.csv/


(['dq-analytics-657082399901-dev/analytics/reports/run_date=2025-11-24/report_best_year.csv/part-00000-73cc26d7-af8b-40b9-b9d1-6e3f028dfd54-c000.csv'],
 ['dq-analytics-657082399901-dev/analytics/reports/run_date=2025-11-24/report_joined.csv/part-00000-4ebb808b-43d8-4436-a90c-1b3e7a8b172a-c000.csv'])

In [11]:
def read_csv_partitioned(file_list):
    dfs = []
    for fpath in file_list:
        print("Reading", fpath)
        dfs.append(pd.read_csv("s3://" + fpath))
    return pd.concat(dfs, ignore_index=True)

df_best = read_csv_partitioned(best_files)
df_joined = read_csv_partitioned(joined_files)

df_best.head(), df_joined.head()


Reading dq-analytics-657082399901-dev/analytics/reports/run_date=2025-11-24/report_best_year.csv/part-00000-73cc26d7-af8b-40b9-b9d1-6e3f028dfd54-c000.csv
Reading dq-analytics-657082399901-dev/analytics/reports/run_date=2025-11-24/report_joined.csv/part-00000-4ebb808b-43d8-4436-a90c-1b3e7a8b172a-c000.csv


(     series_id  best_year  summed_value_for_best_year
 0  PRS30006011     2022.0                      20.500
 1  PRS30006012     2022.0                      17.100
 2  PRS30006013     1989.0                     722.957
 3  PRS30006021     2010.0                      17.700
 4  PRS30006022     2010.0                      12.400,
      series_id  period    year  value  population
 0    series_id  period     NaN    NaN         NaN
 1  PRS30006011     Q01  1988.0    1.9         NaN
 2  PRS30006011     Q02  1988.0    2.2         NaN
 3  PRS30006011     Q03  1988.0    1.9         NaN
 4  PRS30006011     Q04  1988.0    1.1         NaN)

In [12]:
summary_bls = (
    df_ts.groupby("series_id")
    .agg(
        min_year=("year", "min"),
        max_year=("year", "max"),
        n_obs=("year", "count"),
        mean_value=("value", "mean"),
    )
    .reset_index()
)

summary_bls.sort_values("n_obs", ascending=False).head(10)


ArrowInvalid: agg function failed [how->min,dtype->dictionary<values=int32, indices=int32, ordered=0>[pyarrow]]